In [ ]:
("C:/Users/Yusuf/Documents/Delete/dens_df.csv",index=0)
dest_raw_df.to_csv("C:/Users/Yusuf/Documents/Delete/dest_raw_df.csv",index=0)
dens_df.to_csv("C:/Users/Yusuf/Documents/Delete/dens_df.csv",index=0)
path = "C:\\Users\\Yusuf\\Desktop\\GA-Tech\\CSE6242\\Project\\KotoMaterials\\KotoMaterials\\"
data = [json.loads(line) for line in open(path+'dwellTimes2.json', 'r')]
#Convert list of json's into data frame
df = pd.DataFrame(columns = ['seq', 'start_timestamp', 'end_timestamp','vin'])
column_names_dict ={'index':'seq',0:'start_timestamp',1:"end_timestamp"}
for i in range(len(data)):
     temp_df = pd.DataFrame(data[i]['dwell']).transpose().reset_index().rename(columns=column_names_dict)
     temp_df['vin'] = data[i]["_id"]
     df = df.append(temp_df, ignore_index=True)
     #Clean data types
df = df.astype({'seq':'int64'})
df['start_timestamp'] = pd.to_datetime(df['start_timestamp'])
df['end_timestamp'] = pd.to_datetime(df['end_timestamp'])
#Extract additional features
df['arrival_hour'] = df["start_timestamp"].dt.hour
df['arrival_date'] = df["start_timestamp"].dt.date
df['arrival_day'] = df["start_timestamp"].dt.day_name()
df['trip_time'] = (df['end_timestamp'] - df['start_timestamp']).astype('timedelta64[m]')
#Sort and shift data to get additional features
df = df.sort_values(by =['vin','seq'])
df["next_trip_start_timestamp"] = df["start_timestamp"].shift(periods=-1)
df["next_trip_seq"] = df['seq'].shift(periods=-1)
df["dwell_time"] = (df['next_trip_start_timestamp'] - df['end_timestamp']).astype('timedelta64[m]')
df['seq_diff'] = df['next_trip_seq'] - df['seq']
path = "C:\\Users\\Yusuf\\Desktop\\GA-Tech\\CSE6242\\Project\\KotoMaterials\\KotoMaterials\\"
data_dest = [json.loads(line) for line in open(path+'destination.json', 'r')]
df_dest = pd.DataFrame(columns = ['seq', 'Lat', 'Lon','vin'])
column_names_dict ={'index':'seq',0:'Lat',1:"Lon"}
for i in range(len(data_dest)):
     temp_df = pd.DataFrame(data_dest[i]['destination']).transpose().reset_index().rename(columns=column_names_dict)
     temp_df['vin'] = data[i]["_id"]
     df_dest = df_dest.append(temp_df, ignore_index=True)
df_dest['seq'] = df_dest['seq'].astype(int)
df_dest.to_csv("C:/Users/Yusuf/Documents/Delete/dest_raw_all_df.csv")
raw_dest_dwell_df = df_dest.merge(df.reset_index(drop=True),on=['vin','seq'], how='inner')
raw_dest_dwell_viz_df = raw_dest_dwell_df[['latitude','longitude','vin','start_timestamp','end_timestamp','arrival_hour']]
raw_dest_dwell_viz_df.to_csv("C:/Users/Yusuf/Documents/Delete/raw_dest_dwell_viz_df.csv",index=0)